# Task 1 f, Calculating reserve factors against column buckling using Euler-Johnson

## We need from the data file:
- volume of the element
- Iyy calculated with `height_str, width_str, thickness_web, thickness_flange, thickness_skin, stringer_pitch`
- dimensions
- `EulerJohnson(EModulus, I_y, area, length, height_str, thickness_flange, thickness_web, radius, sigma_yield, sigma_applied, c=1)`

## Please enter your name (daniel, fabian, felix, yannis)

In [35]:
name = 'yannis'

## Imports

In [36]:
import pandas as pd
import numpy as np
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.columnbuckling import *
from formulas.panels import *
from formulas.helpers import *
from optimization.generation import *
from optimization.columnBuckReverse import *

In [37]:
# config parser
import configparser

# get the rounding_digits from the ini file
config = configparser.ConfigParser()
config.read('../config.ini')
rounding_digits = int(config['DEFAULT']['rounding_digits'])

## Constants

In [38]:
personal_data = personal_data_provider(name)
sigma_yield = personal_data[0]
EModulus = personal_data[1]
print(f"Your personal data is: sigma_yield = {sigma_yield}, EModulus = {EModulus}. Please verify.")

stringer_pitch = 200
effective_width = stringer_pitch/2

panel_element_length = 750/3
stringer_element_length = 750/3


Your personal data is: sigma_yield = 490, EModulus = 65241.07. Please verify.


# Importing necessary files 

## Import everything for panels 

In [39]:
# Import panel properties and match the elements to the respective panel
paneldf = pd.read_csv(f'../data/{name}/panel.csv')
panelPropertiesdf = pd.read_csv(f'../data/{name}/properties/panel_properties.csv', index_col=0)
panelPropertiesdf = panelPropertiesdf.drop(['mass', 'Component Name'], axis=1)
paneldf = pd.merge(paneldf, panelPropertiesdf, on='Element ID', how='left', suffixes=('_caller', '_other'))
paneldf = paneldf.drop(['sigmaXY', 'sigmaYY'], axis=1)
paneldf.head(5)

,Element ID,Component Name,sigmaXX,Load Case,thickness
0,1,panel1,-56.885,Subcase 1 (LC1),5.00
1,2,panel1,-70.083,Subcase 1 (LC1),5.00
2,3,panel1,-57.534,Subcase 1 (LC1),5.00
3,4,panel2,-76.911,Subcase 1 (LC1),4.94
4,5,panel2,-71.247,Subcase 1 (LC1),4.94


## Add a volume column to the panels 

In [40]:
paneldf['Volume'] = paneldf.apply(panel_element_volume, elementLength=panel_element_length, elementWidth=stringer_pitch, axis=1)
paneldf

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume
0,1,panel1,-56.885,Subcase 1 (LC1),5.00,125000.0
1,2,panel1,-70.083,Subcase 1 (LC1),5.00,125000.0
2,3,panel1,-57.534,Subcase 1 (LC1),5.00,125000.0
3,4,panel2,-76.911,Subcase 1 (LC1),4.94,123500.0
4,5,panel2,-71.247,Subcase 1 (LC1),4.94,123500.0
...,...,...,...,...,...,...
85,26,panel9,-23.669,Subcase 3 (LC3),4.94,123500.0
86,27,panel9,-27.267,Subcase 3 (LC3),4.94,123500.0
87,28,panel10,-20.300,Subcase 3 (LC3),5.00,125000.0
88,29,panel10,-23.283,Subcase 3 (LC3),5.00,125000.0


## Extract thicknesses left and right 

In [41]:
leftThickness = []
rightThickness = []
for i in range(0,9):
    leftThickness.append(paneldf['thickness'][0+3*i])
    rightThickness.append(paneldf['thickness'][3+3*i])
print(leftThickness)
print(rightThickness)

[np.float64(5.0), np.float64(4.94), np.float64(5.01), np.float64(5.29), np.float64(6.02), np.float64(6.02), np.float64(5.29), np.float64(5.01), np.float64(4.94)]
[np.float64(4.94), np.float64(5.01), np.float64(5.29), np.float64(6.02), np.float64(6.02), np.float64(5.29), np.float64(5.01), np.float64(4.94), np.float64(5.0)]


## Import everything for stringers 

In [42]:
# Open and match stringer properties 
stringerdf = pd.read_csv(f'../data/{name}/stringer.csv')
# Now add the stringer properties '../data/{name}/stringer_properties.csv
stringerPropertiesdf = pd.read_csv(f'../data/{name}/properties/stringer_properties.csv', index_col=0)
stringerPropertiesdf = stringerPropertiesdf.reset_index()
stringerPropertiesdf.rename(columns={'beamsects': 'Component Name', 'beamsect_dim1': 'dim1', 'beamsect_dim2': 'dim2', 'beamsect_dim3': 'dim3', 'beamsect_dim4': 'dim4'}, inplace=True)
# Add "stringer" prefix to Component Name
stringerPropertiesdf['Component Name'] = 'stringer' + stringerPropertiesdf['Component Name'].astype(str)
# Merge the dataframes
stringerdf = pd.merge(stringerdf, stringerPropertiesdf, on='Component Name', how='left', suffixes=('_caller', '_other'))
stringerdf

,Element ID,Component Name,sigmaXX,Load Case,dim1,dim2,dim3,dim4
0,40,stringer1,-57.426,Subcase 1 (LC1),36.90,1.69,27.82,15.41
1,41,stringer1,-52.850,Subcase 1 (LC1),36.90,1.69,27.82,15.41
2,42,stringer1,-59.080,Subcase 1 (LC1),36.90,1.69,27.82,15.41
3,43,stringer2,-61.209,Subcase 1 (LC1),38.59,1.51,28.82,14.51
4,44,stringer2,-64.293,Subcase 1 (LC1),38.59,1.51,28.82,14.51
...,...,...,...,...,...,...,...,...
76,62,stringer8,-21.359,Subcase 3 (LC3),38.59,1.51,28.82,14.51
77,63,stringer8,-15.171,Subcase 3 (LC3),38.59,1.51,28.82,14.51
78,64,stringer9,-22.649,Subcase 3 (LC3),36.90,1.69,27.82,15.41
79,65,stringer9,-17.558,Subcase 3 (LC3),36.90,1.69,27.82,15.41


## Add volume to the stringer elements

In [43]:
stringerdf['Volume']= stringerdf.apply(stringer_element_volume, elementLength = stringer_element_length, axis=1)
stringerdf.head(3)

,Element ID,Component Name,sigmaXX,Load Case,dim1,dim2,dim3,dim4,Volume
0,40,stringer1,-57.426,Subcase 1 (LC1),36.9,1.69,27.82,15.41,54527.85
1,41,stringer1,-52.850,Subcase 1 (LC1),36.9,1.69,27.82,15.41,54527.85
2,42,stringer1,-59.080,Subcase 1 (LC1),36.9,1.69,27.82,15.41,54527.85


# Now we rearrange the structure a bit

## First we split the 3 loadcases 

In [44]:
loadCase1dfPanel = paneldf[paneldf["Load Case"] == 'Subcase 1 (LC1)']
loadCase2dfPanel = paneldf[paneldf["Load Case"] == 'Subcase 2 (LC2)']
loadCase3dfPanel = paneldf[paneldf["Load Case"] == 'Subcase 3 (LC3)']
loadCase1dfStringer = stringerdf[stringerdf["Load Case"] == 'Subcase 1 (LC1)']
loadCase2dfStringer = stringerdf[stringerdf["Load Case"] == 'Subcase 2 (LC2)']
loadCase3dfStringer = stringerdf[stringerdf["Load Case"] == 'Subcase 3 (LC3)']
print(loadCase1dfPanel.head(5))
print(loadCase1dfStringer.head(5))

   Element ID Component Name  sigmaXX        Load Case  thickness    Volume
0           1         panel1  -56.885  Subcase 1 (LC1)       5.00  125000.0
1           2         panel1  -70.083  Subcase 1 (LC1)       5.00  125000.0
2           3         panel1  -57.534  Subcase 1 (LC1)       5.00  125000.0
3           4         panel2  -76.911  Subcase 1 (LC1)       4.94  123500.0
4           5         panel2  -71.247  Subcase 1 (LC1)       4.94  123500.0
   Element ID Component Name  sigmaXX        Load Case   dim1  dim2   dim3  \
0          40      stringer1  -57.426  Subcase 1 (LC1)  36.90  1.69  27.82   
1          41      stringer1  -52.850  Subcase 1 (LC1)  36.90  1.69  27.82   
2          42      stringer1  -59.080  Subcase 1 (LC1)  36.90  1.69  27.82   
3          43      stringer2  -61.209  Subcase 1 (LC1)  38.59  1.51  28.82   
4          44      stringer2  -64.293  Subcase 1 (LC1)  38.59  1.51  28.82   

    dim4    Volume  
0  15.41  54527.85  
1  15.41  54527.85  
2  15.41  54

# Now we need to combine the panels and the stringers

In [45]:
# For every loadcase
# Efficiently combine panels for load case 1 
panel_groups_1 = []
for i in range(1, 10):
    df1 = loadCase1dfPanel[loadCase1dfPanel['Component Name'] == f'panel{i}'].copy()
    df2 = loadCase1dfPanel[loadCase1dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df3 = loadCase1dfStringer[loadCase1dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_1.extend([df1, df2, df3])
lc1combined = pd.concat(panel_groups_1, ignore_index=True)
lc1combined.head(10)

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume,Stiffener,dim1,dim2,dim3,dim4
0,1,panel1,-56.885,Subcase 1 (LC1),5.00,125000.00,stiffener1,NaN,NaN,NaN,NaN
1,2,panel1,-70.083,Subcase 1 (LC1),5.00,125000.00,stiffener1,NaN,NaN,NaN,NaN
2,3,panel1,-57.534,Subcase 1 (LC1),5.00,125000.00,stiffener1,NaN,NaN,NaN,NaN
3,4,panel2,-76.911,Subcase 1 (LC1),4.94,123500.00,stiffener1,NaN,NaN,NaN,NaN
4,5,panel2,-71.247,Subcase 1 (LC1),4.94,123500.00,stiffener1,NaN,NaN,NaN,NaN
5,6,panel2,-75.971,Subcase 1 (LC1),4.94,123500.00,stiffener1,NaN,NaN,NaN,NaN
6,40,stringer1,-57.426,Subcase 1 (LC1),NaN,54527.85,stiffener1,36.9,1.69,27.82,15.41
7,41,stringer1,-52.850,Subcase 1 (LC1),NaN,54527.85,stiffener1,36.9,1.69,27.82,15.41
8,42,stringer1,-59.080,Subcase 1 (LC1),NaN,54527.85,stiffener1,36.9,1.69,27.82,15.41
9,4,panel2,-76.911,Subcase 1 (LC1),4.94,123500.00,stiffener2,NaN,NaN,NaN,NaN


In [46]:
# Efficiently combine panels for load case 2
panel_groups_2 = []
for i in range(1, 10):
    df1 = loadCase2dfPanel[loadCase2dfPanel['Component Name'] == f'panel{i}'].copy()
    df2 = loadCase2dfPanel[loadCase2dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df3 = loadCase2dfStringer[loadCase2dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_2.extend([df1, df2, df3])
lc2combined = pd.concat(panel_groups_2, ignore_index=True)
lc2combined.head(10)

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume,Stiffener,dim1,dim2,dim3,dim4
0,1,panel1,-58.628,Subcase 2 (LC2),5.00,125000.00,stiffener1,NaN,NaN,NaN,NaN
1,2,panel1,-70.733,Subcase 2 (LC2),5.00,125000.00,stiffener1,NaN,NaN,NaN,NaN
2,3,panel1,-59.277,Subcase 2 (LC2),5.00,125000.00,stiffener1,NaN,NaN,NaN,NaN
3,4,panel2,-76.257,Subcase 2 (LC2),4.94,123500.00,stiffener1,NaN,NaN,NaN,NaN
4,5,panel2,-69.993,Subcase 2 (LC2),4.94,123500.00,stiffener1,NaN,NaN,NaN,NaN
5,6,panel2,-75.318,Subcase 2 (LC2),4.94,123500.00,stiffener1,NaN,NaN,NaN,NaN
6,40,stringer1,-63.302,Subcase 2 (LC2),NaN,54527.85,stiffener1,36.9,1.69,27.82,15.41
7,41,stringer1,-67.686,Subcase 2 (LC2),NaN,54527.85,stiffener1,36.9,1.69,27.82,15.41
8,42,stringer1,-64.956,Subcase 2 (LC2),NaN,54527.85,stiffener1,36.9,1.69,27.82,15.41
9,4,panel2,-76.257,Subcase 2 (LC2),4.94,123500.00,stiffener2,NaN,NaN,NaN,NaN


In [47]:
# Efficiently combine panels for load case 3 
panel_groups_3 = []
for i in range(1, 10):
    df1 = loadCase3dfPanel[loadCase3dfPanel['Component Name'] == f'panel{i}'].copy()
    df2 = loadCase3dfPanel[loadCase3dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df3 = loadCase3dfStringer[loadCase3dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_3.extend([df1, df2, df3])
lc3combined = pd.concat(panel_groups_3, ignore_index=True)
lc3combined.head(10)

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume,Stiffener,dim1,dim2,dim3,dim4
0,1,panel1,-17.711,Subcase 3 (LC3),5.00,125000.00,stiffener1,NaN,NaN,NaN,NaN
1,2,panel1,-23.283,Subcase 3 (LC3),5.00,125000.00,stiffener1,NaN,NaN,NaN,NaN
2,3,panel1,-20.300,Subcase 3 (LC3),5.00,125000.00,stiffener1,NaN,NaN,NaN,NaN
3,4,panel2,-27.267,Subcase 3 (LC3),4.94,123500.00,stiffener1,NaN,NaN,NaN,NaN
4,5,panel2,-23.669,Subcase 3 (LC3),4.94,123500.00,stiffener1,NaN,NaN,NaN,NaN
5,6,panel2,-23.523,Subcase 3 (LC3),4.94,123500.00,stiffener1,NaN,NaN,NaN,NaN
6,40,stringer1,-16.056,Subcase 3 (LC3),NaN,54527.85,stiffener1,36.9,1.69,27.82,15.41
7,41,stringer1,-17.558,Subcase 3 (LC3),NaN,54527.85,stiffener1,36.9,1.69,27.82,15.41
8,42,stringer1,-22.649,Subcase 3 (LC3),NaN,54527.85,stiffener1,36.9,1.69,27.82,15.41
9,4,panel2,-27.267,Subcase 3 (LC3),4.94,123500.00,stiffener2,NaN,NaN,NaN,NaN


In [48]:
# We fill the empty spaces with zero
lc1combined = lc1combined.fillna(0)
lc2combined = lc2combined.fillna(0)
lc3combined = lc3combined.fillna(0)
lc1combined

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume,Stiffener,dim1,dim2,dim3,dim4
0,1,panel1,-56.885,Subcase 1 (LC1),5.00,125000.00,stiffener1,0.0,0.00,0.00,0.00
1,2,panel1,-70.083,Subcase 1 (LC1),5.00,125000.00,stiffener1,0.0,0.00,0.00,0.00
2,3,panel1,-57.534,Subcase 1 (LC1),5.00,125000.00,stiffener1,0.0,0.00,0.00,0.00
3,4,panel2,-76.911,Subcase 1 (LC1),4.94,123500.00,stiffener1,0.0,0.00,0.00,0.00
4,5,panel2,-71.247,Subcase 1 (LC1),4.94,123500.00,stiffener1,0.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...
76,29,panel10,-70.083,Subcase 1 (LC1),5.00,125000.00,stiffener9,0.0,0.00,0.00,0.00
77,30,panel10,-56.885,Subcase 1 (LC1),5.00,125000.00,stiffener9,0.0,0.00,0.00,0.00
78,64,stringer9,-59.080,Subcase 1 (LC1),0.00,54527.85,stiffener9,36.9,1.69,27.82,15.41
79,65,stringer9,-52.850,Subcase 1 (LC1),0.00,54527.85,stiffener9,36.9,1.69,27.82,15.41


# Now we can aggregate the loadcases according to stiffeners

## Multiply volume and stress together for averaging 

In [49]:
lc1combined['XX * Volume'] = lc1combined['sigmaXX'] * lc1combined['Volume']
lc2combined['XX * Volume'] = lc2combined['sigmaXX'] * lc2combined['Volume']
lc3combined['XX * Volume'] = lc3combined['sigmaXX'] * lc3combined['Volume']
lc1combined.head(9)

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume,Stiffener,dim1,dim2,dim3,dim4,XX * Volume
0,1,panel1,-56.885,Subcase 1 (LC1),5.00,125000.00,stiffener1,0.0,0.00,0.00,0.00,-7.110625e+06
1,2,panel1,-70.083,Subcase 1 (LC1),5.00,125000.00,stiffener1,0.0,0.00,0.00,0.00,-8.760375e+06
2,3,panel1,-57.534,Subcase 1 (LC1),5.00,125000.00,stiffener1,0.0,0.00,0.00,0.00,-7.191750e+06
3,4,panel2,-76.911,Subcase 1 (LC1),4.94,123500.00,stiffener1,0.0,0.00,0.00,0.00,-9.498508e+06
4,5,panel2,-71.247,Subcase 1 (LC1),4.94,123500.00,stiffener1,0.0,0.00,0.00,0.00,-8.799004e+06
5,6,panel2,-75.971,Subcase 1 (LC1),4.94,123500.00,stiffener1,0.0,0.00,0.00,0.00,-9.382418e+06
6,40,stringer1,-57.426,Subcase 1 (LC1),0.00,54527.85,stiffener1,36.9,1.69,27.82,15.41,-3.131316e+06
7,41,stringer1,-52.850,Subcase 1 (LC1),0.00,54527.85,stiffener1,36.9,1.69,27.82,15.41,-2.881797e+06
8,42,stringer1,-59.080,Subcase 1 (LC1),0.00,54527.85,stiffener1,36.9,1.69,27.82,15.41,-3.221505e+06


## Load case 1 

In [50]:
lc1combined = lc1combined.groupby('Stiffener').agg({
    'XX * Volume':'sum',
    'Volume':'sum',
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
})
lc1combined['sigma_XX_avg'] = lc1combined['XX * Volume'] / lc1combined['Volume']
lc1combined['tLeft'] = leftThickness
lc1combined['tRight'] = rightThickness
lc1combined = lc1combined.drop(['XX * Volume'], axis=1)
lc1combined

,Volume,dim1,dim2,dim3,dim4,sigma_XX_avg,tLeft,tRight
Stiffener,,,,,,,,
stiffener1,909083.550,36.90,1.69,27.82,15.41,-65.975564,5.00,4.94
stiffener2,895740.000,38.59,1.51,28.82,14.51,-72.343316,4.94,5.01
stiffener3,929182.575,37.42,1.59,29.05,15.34,-71.632511,5.01,5.29
stiffener4,1007645.850,37.84,1.62,28.83,14.96,-70.705864,5.29,6.02
stiffener5,1055089.650,37.80,1.53,29.90,15.05,-70.321610,6.02,6.02
stiffener6,1007645.850,37.84,1.62,28.83,14.96,-70.705864,6.02,5.29
stiffener7,929182.575,37.42,1.59,29.05,15.34,-71.632511,5.29,5.01
stiffener8,895740.000,38.59,1.51,28.82,14.51,-72.343316,5.01,4.94
stiffener9,909083.550,36.90,1.69,27.82,15.41,-65.975564,4.94,5.00


## Load case 2

In [51]:
lc2combined = lc2combined.groupby('Stiffener').agg({
    'XX * Volume':'sum',
    'Volume':'sum',
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
})
lc2combined['sigma_XX_avg'] = lc2combined['XX * Volume'] / lc2combined['Volume']
lc2combined = lc2combined.drop(['XX * Volume'], axis=1)
lc2combined['tLeft'] = leftThickness
lc2combined['tRight'] = rightThickness
lc2combined

,Volume,dim1,dim2,dim3,dim4,sigma_XX_avg,tLeft,tRight
Stiffener,,,,,,,,
stiffener1,909083.550,36.90,1.69,27.82,15.41,-67.791132,5.00,4.94
stiffener2,895740.000,38.59,1.51,28.82,14.51,-71.354522,4.94,5.01
stiffener3,929182.575,37.42,1.59,29.05,15.34,-71.440887,5.01,5.29
stiffener4,1007645.850,37.84,1.62,28.83,14.96,-70.905553,5.29,6.02
stiffener5,1055089.650,37.80,1.53,29.90,15.05,-70.501912,6.02,6.02
stiffener6,1007645.850,37.84,1.62,28.83,14.96,-70.905553,6.02,5.29
stiffener7,929182.575,37.42,1.59,29.05,15.34,-71.440887,5.29,5.01
stiffener8,895740.000,38.59,1.51,28.82,14.51,-71.354522,5.01,4.94
stiffener9,909083.550,36.90,1.69,27.82,15.41,-67.791132,4.94,5.00


## Load case 3

In [52]:
lc3combined = lc3combined.groupby('Stiffener').agg({
    'XX * Volume':'sum',
    'Volume':'sum',
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
})
lc3combined['sigma_XX_avg'] = lc3combined['XX * Volume'] / lc3combined['Volume']
lc3combined = lc3combined.drop(['XX * Volume'], axis=1)
lc3combined['tLeft'] = leftThickness
lc3combined['tRight'] = rightThickness
lc3combined

,Volume,dim1,dim2,dim3,dim4,sigma_XX_avg,tLeft,tRight
Stiffener,,,,,,,,
stiffener1,909083.550,36.90,1.69,27.82,15.41,-21.918048,5.00,4.94
stiffener2,895740.000,38.59,1.51,28.82,14.51,-24.033547,4.94,5.01
stiffener3,929182.575,37.42,1.59,29.05,15.34,-23.797290,5.01,5.29
stiffener4,1007645.850,37.84,1.62,28.83,14.96,-23.489568,5.29,6.02
stiffener5,1055089.650,37.80,1.53,29.90,15.05,-23.361979,6.02,6.02
stiffener6,1007645.850,37.84,1.62,28.83,14.96,-23.489568,6.02,5.29
stiffener7,929182.575,37.42,1.59,29.05,15.34,-23.797290,5.29,5.01
stiffener8,895740.000,38.59,1.51,28.82,14.51,-24.033547,5.01,4.94
stiffener9,909083.550,36.90,1.69,27.82,15.41,-21.918048,4.94,5.00


# Now we add Cross-Section Properties of the combined skin and hat stringer crosssection 

In [53]:
# Load case 1
lc1combined[['I_yy', 'areaTot', 'VolumeTot']] = lc1combined.apply(crosssectional_properties_hat_skin_row, stringer_pitch=stringer_pitch, stringer_depth = stringer_element_length*3,
                                                                    axis=1, result_type='expand')
# Load case 2
lc2combined[['I_yy', 'areaTot', 'VolumeTot']] = lc2combined.apply(crosssectional_properties_hat_skin_row, stringer_pitch=stringer_pitch, stringer_depth = stringer_element_length*3,
                                                                    axis=1, result_type='expand')
# Load case 3
lc3combined[['I_yy', 'areaTot', 'VolumeTot']] = lc3combined.apply(crosssectional_properties_hat_skin_row, stringer_pitch=stringer_pitch, stringer_depth = stringer_element_length*3,
                                                                    axis=1, result_type='expand')
lc1combined

,Volume,dim1,dim2,dim3,dim4,sigma_XX_avg,tLeft,tRight,I_yy,areaTot,VolumeTot
Stiffener,,,,,,,,,,,
stiffener1,909083.550,36.90,1.69,27.82,15.41,-65.975564,5.00,4.94,117116.833425,1212.1114,909083.550
stiffener2,895740.000,38.59,1.51,28.82,14.51,-72.343316,4.94,5.01,120371.549642,1194.3200,895740.000
stiffener3,929182.575,37.42,1.59,29.05,15.34,-71.632511,5.01,5.29,118365.891813,1238.9101,929182.575
stiffener4,1007645.850,37.84,1.62,28.83,14.96,-70.705864,5.29,6.02,125770.822449,1343.5278,1007645.850
stiffener5,1055089.650,37.80,1.53,29.90,15.05,-70.321610,6.02,6.02,125993.523988,1406.7862,1055089.650
stiffener6,1007645.850,37.84,1.62,28.83,14.96,-70.705864,6.02,5.29,128533.054673,1343.5278,1007645.850
stiffener7,929182.575,37.42,1.59,29.05,15.34,-71.632511,5.29,5.01,119369.112296,1238.9101,929182.575
stiffener8,895740.000,38.59,1.51,28.82,14.51,-72.343316,5.01,4.94,120619.489851,1194.3200,895740.000
stiffener9,909083.550,36.90,1.69,27.82,15.41,-65.975564,4.94,5.00,116901.502550,1212.1114,909083.550


# Now we calculate the columnbuckling with Euler Johnson 

In [54]:
# Load case 1EModulus, DIM1, DIM2, DIM3, sigma_yield, r
lc1combined['sigma_crip'] = lc1combined.apply(lambda row: sigma_crip(EModulus= EModulus, DIM1=row['dim1'], DIM2=row['dim2'], DIM3=row['dim3'], sigma_yield = sigma_yield, r=0 ),
                                                axis=1)
# Load case 2
lc2combined['sigma_crip'] = lc2combined.apply(lambda row: sigma_crip(EModulus= EModulus, DIM1=row['dim1'], DIM2=row['dim2'], DIM3=row['dim3'], sigma_yield = sigma_yield, r=0 ),
                                                axis=1)
#Load case 3
lc3combined['sigma_crip'] = lc3combined.apply(lambda row: sigma_crip(EModulus= EModulus, DIM1=row['dim1'], DIM2=row['dim2'], DIM3=row['dim3'], sigma_yield = sigma_yield, r=0 ),
                                                axis=1)
lc1combined

,Volume,dim1,dim2,dim3,dim4,sigma_XX_avg,tLeft,tRight,I_yy,areaTot,VolumeTot,sigma_crip
Stiffener,,,,,,,,,,,,
stiffener1,909083.550,36.90,1.69,27.82,15.41,-65.975564,5.00,4.94,117116.833425,1212.1114,909083.550,408.093629
stiffener2,895740.000,38.59,1.51,28.82,14.51,-72.343316,4.94,5.01,120371.549642,1194.3200,895740.000,359.901302
stiffener3,929182.575,37.42,1.59,29.05,15.34,-71.632511,5.01,5.29,118365.891813,1238.9101,929182.575,384.331012
stiffener4,1007645.850,37.84,1.62,28.83,14.96,-70.705864,5.29,6.02,125770.822449,1343.5278,1007645.850,387.614361
stiffener5,1055089.650,37.80,1.53,29.90,15.05,-70.321610,6.02,6.02,125993.523988,1406.7862,1055089.650,367.757298
stiffener6,1007645.850,37.84,1.62,28.83,14.96,-70.705864,6.02,5.29,128533.054673,1343.5278,1007645.850,387.614361
stiffener7,929182.575,37.42,1.59,29.05,15.34,-71.632511,5.29,5.01,119369.112296,1238.9101,929182.575,384.331012
stiffener8,895740.000,38.59,1.51,28.82,14.51,-72.343316,5.01,4.94,120619.489851,1194.3200,895740.000,359.901302
stiffener9,909083.550,36.90,1.69,27.82,15.41,-65.975564,4.94,5.00,116901.502550,1212.1114,909083.550,408.093629


## Add lambda_crit to the loadcases

In [55]:
lc1combined['lambda_crit'] = lc1combined.apply(lambda row: lambda_crit(EModulus, row['sigma_crip'], personal_data[0]), axis=1)
lc2combined['lambda_crit'] = lc2combined.apply(lambda row: lambda_crit(EModulus, row['sigma_crip'], personal_data[0]), axis=1)
lc3combined['lambda_crit'] = lc3combined.apply(lambda row: lambda_crit(EModulus, row['sigma_crip'], personal_data[0]), axis=1)

## Add lambda to the loadcases

In [56]:
lc1combined['lambda'] = lc1combined.apply(lambda row: lmd(row['I_yy'], row['areaTot'], stringer_element_length*3), axis=1)
lc2combined['lambda'] = lc2combined.apply(lambda row: lmd(row['I_yy'], row['areaTot'], stringer_element_length*3), axis=1)
lc3combined['lambda'] = lc3combined.apply(lambda row: lmd(row['I_yy'], row['areaTot'], stringer_element_length*3), axis=1)

## Add radius of gyration to the loadcases

In [57]:
lc1combined['r_gyr'] = lc1combined.apply(lambda row: r_gyr(row['I_yy'], row['areaTot']), axis=1)
lc2combined['r_gyr'] = lc2combined.apply(lambda row: r_gyr(row['I_yy'], row['areaTot']), axis=1)
lc3combined['r_gyr'] = lc3combined.apply(lambda row: r_gyr(row['I_yy'], row['areaTot']), axis=1)

## Now calculate the critical stress & Reserve Factor

In [58]:
lc1combined[['sigma_crit', 'Reserve Factor']] = lc1combined.apply(chooseBuckling, EModulus=EModulus, sigma_yield=sigma_yield, axis=1, result_type='expand')
lc2combined[['sigma_crit', 'Reserve Factor']] = lc2combined.apply(chooseBuckling, EModulus=EModulus, sigma_yield=sigma_yield, axis=1, result_type='expand')
lc3combined[['sigma_crit', 'Reserve Factor']] = lc3combined.apply(chooseBuckling, EModulus=EModulus, sigma_yield=sigma_yield, axis=1, result_type='expand')
lc1combined

,Volume,dim1,dim2,dim3,dim4,sigma_XX_avg,tLeft,tRight,I_yy,areaTot,VolumeTot,sigma_crip,lambda_crit,lambda,r_gyr,sigma_crit,Reserve Factor
Stiffener,,,,,,,,,,,,,,,,,
stiffener1,909083.550,36.90,1.69,27.82,15.41,-65.975564,5.00,4.94,117116.833425,1212.1114,909083.550,408.093629,56.175313,76.299707,9.829658,110.605081,1.117637
stiffener2,895740.000,38.59,1.51,28.82,14.51,-72.343316,4.94,5.01,120371.549642,1194.3200,895740.000,359.901302,59.818249,74.706724,10.039257,115.372271,1.063192
stiffener3,929182.575,37.42,1.59,29.05,15.34,-71.632511,5.01,5.29,118365.891813,1238.9101,929182.575,384.331012,57.885887,76.730473,9.774474,109.366692,1.017850
stiffener4,1007645.850,37.84,1.62,28.83,14.96,-70.705864,5.29,6.02,125770.822449,1343.5278,1007645.850,387.614361,57.640200,77.516587,9.675349,107.159712,1.010380
stiffener5,1055089.650,37.80,1.53,29.90,15.05,-70.321610,6.02,6.02,125993.523988,1406.7862,1055089.650,367.757298,59.175884,79.250352,9.463680,102.522318,0.971938
stiffener6,1007645.850,37.84,1.62,28.83,14.96,-70.705864,6.02,5.29,128533.054673,1343.5278,1007645.850,387.614361,57.640200,76.679130,9.781019,109.513199,1.032571
stiffener7,929182.575,37.42,1.59,29.05,15.34,-71.632511,5.29,5.01,119369.112296,1238.9101,929182.575,384.331012,57.885887,76.407357,9.815809,110.293639,1.026477
stiffener8,895740.000,38.59,1.51,28.82,14.51,-72.343316,5.01,4.94,120619.489851,1194.3200,895740.000,359.901302,59.818249,74.629902,10.049591,115.609913,1.065382
stiffener9,909083.550,36.90,1.69,27.82,15.41,-65.975564,4.94,5.00,116901.502550,1212.1114,909083.550,408.093629,56.175313,76.369946,9.820617,110.401723,1.115582


# Generate Score output 

In [59]:

lc1combined = lc1combined.reset_index()
lc2combined = lc2combined.reset_index()
lc3combined = lc3combined.reset_index()

evaluateDf = pd.concat([lc1combined, lc2combined, lc3combined])
evaluateDf = evaluateDf.reset_index()
evaluateDf = evaluateDf.drop(['index'], axis=1)
updateDF = evaluateDf.copy()
evaluateDf

,Stiffener,Volume,dim1,dim2,dim3,dim4,sigma_XX_avg,tLeft,tRight,I_yy,areaTot,VolumeTot,sigma_crip,lambda_crit,lambda,r_gyr,sigma_crit,Reserve Factor
0,stiffener1,909083.550,36.90,1.69,27.82,15.41,-65.975564,5.00,4.94,117116.833425,1212.1114,909083.550,408.093629,56.175313,76.299707,9.829658,110.605081,1.117637
1,stiffener2,895740.000,38.59,1.51,28.82,14.51,-72.343316,4.94,5.01,120371.549642,1194.3200,895740.000,359.901302,59.818249,74.706724,10.039257,115.372271,1.063192
2,stiffener3,929182.575,37.42,1.59,29.05,15.34,-71.632511,5.01,5.29,118365.891813,1238.9101,929182.575,384.331012,57.885887,76.730473,9.774474,109.366692,1.017850
3,stiffener4,1007645.850,37.84,1.62,28.83,14.96,-70.705864,5.29,6.02,125770.822449,1343.5278,1007645.850,387.614361,57.640200,77.516587,9.675349,107.159712,1.010380
4,stiffener5,1055089.650,37.80,1.53,29.90,15.05,-70.321610,6.02,6.02,125993.523988,1406.7862,1055089.650,367.757298,59.175884,79.250352,9.463680,102.522318,0.971938
5,stiffener6,1007645.850,37.84,1.62,28.83,14.96,-70.705864,6.02,5.29,128533.054673,1343.5278,1007645.850,387.614361,57.640200,76.679130,9.781019,109.513199,1.032571
6,stiffener7,929182.575,37.42,1.59,29.05,15.34,-71.632511,5.29,5.01,119369.112296,1238.9101,929182.575,384.331012,57.885887,76.407357,9.815809,110.293639,1.026477
7,stiffener8,895740.000,38.59,1.51,28.82,14.51,-72.343316,5.01,4.94,120619.489851,1194.3200,895740.000,359.901302,59.818249,74.629902,10.049591,115.609913,1.065382
8,stiffener9,909083.550,36.90,1.69,27.82,15.41,-65.975564,4.94,5.00,116901.502550,1212.1114,909083.550,408.093629,56.175313,76.369946,9.820617,110.401723,1.115582
9,stiffener1,909083.550,36.90,1.69,27.82,15.41,-67.791132,5.00,4.94,117116.833425,1212.1114,909083.550,408.093629,56.175313,76.299707,9.829658,110.605081,1.087705


In [60]:
evaluateDf = evaluateDf.drop(['Volume' ,'tLeft' ,'tRight', 'sigma_XX_avg', 'I_yy',  'areaTot', 'VolumeTot', 'sigma_crip', 'lambda_crit', 'lambda', 'r_gyr', 'sigma_crit'], axis =1)
evaluateDf['score'] = evaluateDf.apply(rf_score, axis=1)
evaluateDf

,Stiffener,dim1,dim2,dim3,dim4,Reserve Factor,score
0,stiffener1,36.90,1.69,27.82,15.41,1.117637,0.114369
1,stiffener2,38.59,1.51,28.82,14.51,1.063192,0.004351
2,stiffener3,37.42,1.59,29.05,15.34,1.017850,0.025841
3,stiffener4,37.84,1.62,28.83,14.96,1.010380,0.039243
4,stiffener5,37.80,1.53,29.90,15.05,0.971938,1523.437414
5,stiffener6,37.84,1.62,28.83,14.96,1.032571,0.007595
6,stiffener7,37.42,1.59,29.05,15.34,1.026477,0.013834
7,stiffener8,38.59,1.51,28.82,14.51,1.065382,0.005915
8,stiffener9,36.90,1.69,27.82,15.41,1.115582,0.107525
9,stiffener1,36.90,1.69,27.82,15.41,1.087705,0.035541


## Extract score and parameters

In [61]:
score = evaluateDf['score'].sum()
dimensions = extractDimensions(evaluateDf)

evaluateDf = pd.DataFrame({
    'stringer Parameters': [dimensions],
    'score': [score],
    'minRF':[evaluateDf['Reserve Factor'].min()]
})
evaluateDf

,stringer Parameters,score,minRF
0,"[[36.9, 1.69, 27.82, 15.41], [38.59, 1.51, 28....",4143.30853,0.969452


## Output stringer score

In [62]:
evaluateDf.to_csv(f'../data/{name}/output/stringerScore.csv', index=False)

# Reverse Engineering 

## Collect all dimension updates 

In [63]:

updateDF[['new_dim1', 'new_dim3']] = updateDF.apply(reverseAllDims, EModulus=EModulus,
                                                                            stringerPitch=stringer_pitch,
                                                                            length=stringer_element_length*3,
                                                                             RFgoal=0.9, axis=1, result_type='expand')
updateDF

,Stiffener,Volume,dim1,dim2,dim3,dim4,sigma_XX_avg,tLeft,tRight,I_yy,areaTot,VolumeTot,sigma_crip,lambda_crit,lambda,r_gyr,sigma_crit,Reserve Factor,new_dim1,new_dim3
0,stiffener1,909083.550,36.90,1.69,27.82,15.41,-65.975564,5.00,4.94,117116.833425,1212.1114,909083.550,408.093629,56.175313,76.299707,9.829658,110.605081,1.117637,27.5807714949806,1.53748316023900
1,stiffener2,895740.000,38.59,1.51,28.82,14.51,-72.343316,4.94,5.01,120371.549642,1194.3200,895740.000,359.901302,59.818249,74.706724,10.039257,115.372271,1.063192,29.4701644637412,2.74142843037734
2,stiffener3,929182.575,37.42,1.59,29.05,15.34,-71.632511,5.01,5.29,118365.891813,1238.9101,929182.575,384.331012,57.885887,76.730473,9.774474,109.366692,1.017850,28.9337284399096,4.13185471287900
3,stiffener4,1007645.850,37.84,1.62,28.83,14.96,-70.705864,5.29,6.02,125770.822449,1343.5278,1007645.850,387.614361,57.640200,77.516587,9.675349,107.159712,1.010380,28.9744840805012,3.26481259958461
4,stiffener5,1055089.650,37.80,1.53,29.90,15.05,-70.321610,6.02,6.02,125993.523988,1406.7862,1055089.650,367.757298,59.175884,79.250352,9.463680,102.522318,0.971938,29.9946748729808,6.48702956601247
5,stiffener6,1007645.850,37.84,1.62,28.83,14.96,-70.705864,6.02,5.29,128533.054673,1343.5278,1007645.850,387.614361,57.640200,76.679130,9.781019,109.513199,1.032571,29.7851844264885,5.25150510086401
6,stiffener7,929182.575,37.42,1.59,29.05,15.34,-71.632511,5.29,5.01,119369.112296,1238.9101,929182.575,384.331012,57.885887,76.407357,9.815809,110.293639,1.026477,29.2863364593490,5.00577140242485
7,stiffener8,895740.000,38.59,1.51,28.82,14.51,-72.343316,5.01,4.94,120619.489851,1194.3200,895740.000,359.901302,59.818249,74.629902,10.049591,115.609913,1.065382,29.5659673572735,2.96747194702377
8,stiffener9,909083.550,36.90,1.69,27.82,15.41,-65.975564,4.94,5.00,116901.502550,1212.1114,909083.550,408.093629,56.175313,76.369946,9.820617,110.401723,1.115582,27.5090583776452,1.37342314920554
9,stiffener1,909083.550,36.90,1.69,27.82,15.41,-67.791132,5.00,4.94,117116.833425,1212.1114,909083.550,408.093629,56.175313,76.299707,9.829658,110.605081,1.087705,27.9443981593723,2.32309341558019


## Cleanup dataframe

In [64]:
updateDF = updateDF.drop(['Volume' ,'tLeft' , 'tRight', 'sigma_XX_avg', 'I_yy',  'areaTot', 'VolumeTot', 'sigma_crip', 'lambda_crit', 'lambda', 'r_gyr', 'sigma_crit'], axis =1)

## Keep only the largest values 

In [65]:
updateDF = updateDF.groupby('Stiffener').agg({
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
    'new_dim1': 'max',
    'new_dim3': 'max'
})
updateDF


,dim1,dim2,dim3,dim4,new_dim1,new_dim3
Stiffener,,,,,,
stiffener1,36.90,1.69,27.82,15.41,27.9443981593723,2.32309341558019
stiffener2,38.59,1.51,28.82,14.51,29.4701644637412,2.74142843037734
stiffener3,37.42,1.59,29.05,15.34,28.9337284399096,4.13185471287900
stiffener4,37.84,1.62,28.83,14.96,29.0140305794056,3.35357033653611
stiffener5,37.80,1.53,29.90,15.05,30.0324850077556,6.57892256150837
stiffener6,37.84,1.62,28.83,14.96,29.8264452737485,5.34846177770685
stiffener7,37.42,1.59,29.05,15.34,29.2863364593490,5.00577140242485
stiffener8,38.59,1.51,28.82,14.51,29.5659673572735,2.96747194702377
stiffener9,36.90,1.69,27.82,15.41,27.8713382439110,2.15262219061199


## Add differnces 

In [66]:
updateDF['diff_dim1'] = (updateDF['new_dim1'] - updateDF['dim1']).astype(float)
updateDF['diff_dim3'] = (updateDF['new_dim3'] - updateDF['dim3']). astype(float)


#Convert type 
updateDF['new_dim1'] = updateDF['new_dim1'].astype(float)
updateDF['new_dim3'] = updateDF['new_dim3'].astype(float)



updateDF = updateDF.round(rounding_digits)
updateDF

,dim1,dim2,dim3,dim4,new_dim1,new_dim3,diff_dim1,diff_dim3
Stiffener,,,,,,,,
stiffener1,36.90,1.69,27.82,15.41,27.94,2.32,-8.96,-25.50
stiffener2,38.59,1.51,28.82,14.51,29.47,2.74,-9.12,-26.08
stiffener3,37.42,1.59,29.05,15.34,28.93,4.13,-8.49,-24.92
stiffener4,37.84,1.62,28.83,14.96,29.01,3.35,-8.83,-25.48
stiffener5,37.80,1.53,29.90,15.05,30.03,6.58,-7.77,-23.32
stiffener6,37.84,1.62,28.83,14.96,29.83,5.35,-8.01,-23.48
stiffener7,37.42,1.59,29.05,15.34,29.29,5.01,-8.13,-24.04
stiffener8,38.59,1.51,28.82,14.51,29.57,2.97,-9.02,-25.85
stiffener9,36.90,1.69,27.82,15.41,27.87,2.15,-9.03,-25.67


## Output of Reverse 

In [67]:
updateDF.to_csv(f'../data/{name}/output/newStringerDims.csv', index=False)

# Cleanup data for output 

## Drop unenessacry columns 

In [68]:
lc1combined = lc1combined.drop(['Volume', 'tLeft', 'tRight', 'dim1', 'dim2', 'dim3', 'dim4', 'areaTot', 'VolumeTot'], axis=1)
lc2combined = lc2combined.drop(['Volume',  'tLeft', 'tRight', 'dim1', 'dim2', 'dim3', 'dim4', 'areaTot', 'VolumeTot'], axis=1)
lc3combined = lc3combined.drop(['Volume',  'tLeft', 'tRight', 'dim1', 'dim2', 'dim3', 'dim4', 'areaTot', 'VolumeTot'], axis=1)
lc1combined

,Stiffener,sigma_XX_avg,I_yy,sigma_crip,lambda_crit,lambda,r_gyr,sigma_crit,Reserve Factor
0,stiffener1,-65.975564,117116.833425,408.093629,56.175313,76.299707,9.829658,110.605081,1.117637
1,stiffener2,-72.343316,120371.549642,359.901302,59.818249,74.706724,10.039257,115.372271,1.063192
2,stiffener3,-71.632511,118365.891813,384.331012,57.885887,76.730473,9.774474,109.366692,1.017850
3,stiffener4,-70.705864,125770.822449,387.614361,57.640200,77.516587,9.675349,107.159712,1.010380
4,stiffener5,-70.321610,125993.523988,367.757298,59.175884,79.250352,9.463680,102.522318,0.971938
5,stiffener6,-70.705864,128533.054673,387.614361,57.640200,76.679130,9.781019,109.513199,1.032571
6,stiffener7,-71.632511,119369.112296,384.331012,57.885887,76.407357,9.815809,110.293639,1.026477
7,stiffener8,-72.343316,120619.489851,359.901302,59.818249,74.629902,10.049591,115.609913,1.065382
8,stiffener9,-65.975564,116901.502550,408.093629,56.175313,76.369946,9.820617,110.401723,1.115582


## ROUND & Add together the load cases 

In [69]:
# Rename colums for concat
lc1combined = lc1combined.rename(columns={'sigma_XX_avg':'XX_avg_LC1', 'sigma_crit':'sigma_crit_LC1', 'Reserve Factor':'RF_LC1', 'sigma_crip':'sigma_crip_LC1'})
lc2combined = lc2combined.rename(columns={'sigma_XX_avg':'XX_avg_LC2', 'sigma_crit':'sigma_crit_LC2', 'Reserve Factor':'RF_LC2', 'sigma_crip':'sigma_crip_LC2'})
lc3combined = lc3combined.rename(columns={'sigma_XX_avg':'XX_avg_LC3', 'sigma_crit':'sigma_crit_LC3', 'Reserve Factor':'RF_LC3', 'sigma_crip':'sigma_crip_LC3'})

In [70]:
outputdf = pd.concat([lc1combined,lc2combined,lc3combined], axis = 1)
outputdf = outputdf.round(rounding_digits)
# After concatenation, keep only the first column of cross section propertries and drop the rest
outputdf['I_y'] = outputdf.filter(like='I_yy').iloc[:, 0]  # Take the first I_yy column
outputdf['Lambda'] = outputdf.filter(like='lambda').iloc[:, 1]  # Take the first lambda column
outputdf['Lambda_crit'] = outputdf.filter(like='lambda_crit').iloc[:, 0]  # Take the first I_yy column
outputdf['R_gyr'] = outputdf.filter(like='r_gyr').iloc[:, 0]  # Take the first I_yy column
outputdf = outputdf.drop(columns=[col for col in outputdf.columns if col.startswith('I_yy')])
outputdf = outputdf.drop(columns=[col for col in outputdf.columns if col.startswith('lambda')])
outputdf = outputdf.drop(columns=[col for col in outputdf.columns if col.startswith('r_gyr')])
outputdf.head(10)

,Stiffener,XX_avg_LC1,sigma_crip_LC1,sigma_crit_LC1,RF_LC1,Stiffener,XX_avg_LC2,sigma_crip_LC2,sigma_crit_LC2,RF_LC2,Stiffener,XX_avg_LC3,sigma_crip_LC3,sigma_crit_LC3,RF_LC3,I_y,Lambda,Lambda_crit,R_gyr
0,stiffener1,-65.98,408.09,110.61,1.12,stiffener1,-67.79,408.09,110.61,1.09,stiffener1,-21.92,408.09,110.61,3.36,117116.83,76.30,56.18,9.83
1,stiffener2,-72.34,359.90,115.37,1.06,stiffener2,-71.35,359.90,115.37,1.08,stiffener2,-24.03,359.90,115.37,3.20,120371.55,74.71,59.82,10.04
2,stiffener3,-71.63,384.33,109.37,1.02,stiffener3,-71.44,384.33,109.37,1.02,stiffener3,-23.80,384.33,109.37,3.06,118365.89,76.73,57.89,9.77
3,stiffener4,-70.71,387.61,107.16,1.01,stiffener4,-70.91,387.61,107.16,1.01,stiffener4,-23.49,387.61,107.16,3.04,125770.82,77.52,57.64,9.68
4,stiffener5,-70.32,367.76,102.52,0.97,stiffener5,-70.50,367.76,102.52,0.97,stiffener5,-23.36,367.76,102.52,2.93,125993.52,79.25,59.18,9.46
5,stiffener6,-70.71,387.61,109.51,1.03,stiffener6,-70.91,387.61,109.51,1.03,stiffener6,-23.49,387.61,109.51,3.11,128533.05,76.68,57.64,9.78
6,stiffener7,-71.63,384.33,110.29,1.03,stiffener7,-71.44,384.33,110.29,1.03,stiffener7,-23.80,384.33,110.29,3.09,119369.11,76.41,57.89,9.82
7,stiffener8,-72.34,359.90,115.61,1.07,stiffener8,-71.35,359.90,115.61,1.08,stiffener8,-24.03,359.90,115.61,3.21,120619.49,74.63,59.82,10.05
8,stiffener9,-65.98,408.09,110.40,1.12,stiffener9,-67.79,408.09,110.40,1.09,stiffener9,-21.92,408.09,110.40,3.36,116901.50,76.37,56.18,9.82


# Generate output file 

In [71]:
outputdf.to_excel(f'../data/{name}/output/processed_f.xlsx')